In [2]:
"Hare krishna"

'Hare krishna'

In [3]:
pip install pytesseract


In [4]:
pip install pillow


In [5]:
pip install easyocr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 21.1 MB/s eta 0:00:00


In [6]:
pip install pandas


In [7]:
import pytesseract
from PIL import Image
import easyocr
import pandas as pd
import re
from datetime import datetime

print("All modules are correctly installed.")


All modules are correctly installed.


In [10]:
!apt-get update
!apt-get install tesseract-ocr


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,190 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,625 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [11]:
import pytesseract
from PIL import Image
import easyocr
import re
import pandas as pd
from datetime import datetime

# Input and output file paths
input_image = "/content/5_page-0001.jpg"  # Replace with your image file path
output_csv = "output.csv"

# Define regions for OCR (x1, y1, x2, y2)
FIELD_POSITIONS = {
    "Name of beneficiary": (515, 491, 817, 523),
    "Mobile": (554, 791, 914, 876),
    "Date": (1118, 243, 1365, 311),  # Coordinates for Date field
    "Record No.": (273, 246, 446, 295),
}

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Function to correct OCR misinterpretations in dates
def correct_ocr_date(ocr_output):
    corrections = {
        "O": "0", "I": "1", "l": "1", "Z": "2", "S": "5", "B": "8", "G": "6",
        "q": "9", "ozz": "022", "zz": "22", "on": "01", "iO": "10", "O5": "05", "lo": "10"
    }

    corrected_output = ocr_output
    while True:
        previous_output = corrected_output
        for wrong, right in corrections.items():
            corrected_output = corrected_output.replace(wrong, right)

        if corrected_output == previous_output:
            break

    return corrected_output

# Function to convert date into DD/MM/YYYY format
def convert_to_ddmmyyyy_format(extracted_date):
    cleaned_date = re.sub(r'[^a-zA-Z0-9\s-]', '', extracted_date)
    cleaned_date = correct_ocr_date(cleaned_date)

    month_mapping = {
        "Jan": "01", "Feb": "02", "Mar": "03", "Apr": "04", "May": "05", "Jun": "06",
        "Jul": "07", "Aug": "08", "Sep": "09", "Oct": "10", "Nov": "11", "Dec": "12"
    }

    date_parts = cleaned_date.split('-')

    if len(date_parts) == 3:
        day = date_parts[0]
        month_str = date_parts[1]
        year = date_parts[2]

        month = month_mapping.get(month_str, "00")

        if len(day) == 2 and len(year) == 4:
            return f"{day}/{month}/{year}"
    return extracted_date

# Function to extract text from image regions using EasyOCR
def extract_text_from_image(image_path, field_positions):
    image = Image.open(image_path)

    # Correct image orientation before extracting text
    result = pytesseract.image_to_osd(image)
    if 'Rotate: 180' in result:
        image = image.rotate(180, expand=True)
        image.save("corrected_image.jpg")
        print("The image was upside down and has been corrected.")

    extracted_data = {}

    for field_name, coords in field_positions.items():
        cropped_region = image.crop(coords)
        cropped_region.save(f'{field_name}.jpg')  # Save cropped region for debugging

        result = reader.readtext(f'{field_name}.jpg')  # Get OCR result from the region
        text = " ".join([item[1] for item in result])

        if field_name == "Mobile":
            text = improve_number_detection(text)

        if field_name == "Date":
            text = correct_ocr_date(text)
            text = convert_to_ddmmyyyy_format(text)

        extracted_data[field_name] = text.strip()

    return extracted_data

# Function to improve number extraction for mobile numbers
def improve_number_detection(extracted_text):
    digits_only = re.sub(r'\D', '', extracted_text)

    if len(digits_only) < 10:
        digits_only = '8' + digits_only  # Assuming '8' as the prefix for India
    return digits_only

# Save extracted data to CSV
def save_to_csv(data, output_path):
    formatted_data = {
        "File name": "image.jpg",  # Replace with your image name
        "Name of beneficiary": data.get("Name of beneficiary", ""),
        "Record No.": data.get("Record No.", ""),
        "Date of document": convert_to_ddmmyyyy_format(data.get("Date", "")),
        "Mobile": data.get("Mobile", ""),
    }
    df = pd.DataFrame([formatted_data])
    print(df)
    df.to_csv(output_path, index=False)

# Process the image and generate output CSV
def process_image(input_image, field_positions, output_csv):
    extracted_data = extract_text_from_image(input_image, field_positions)
    save_to_csv(extracted_data, output_csv)

# Run the process
process_image(input_image, FIELD_POSITIONS, output_csv)


   File name  Name of beneficiary Record No. Date of document      Mobile
0  image.jpg  GURJAR CONSTRUCTION       4507       20/08/2022  7724472970
